In [1]:
import numpy as np
#create a class in which acts as a base for different algorithms. 
#specifically the train and predict steps of different models. 
#inputdata = the input data 
#output = the output and not  really used in unsupervised alogorithms 
#**kwargs lets it use parameters without having to state them 
class MachineLearningALgos:
    def train(self, inputdata, output, **kwargs):
        raise NotImplementedError("Check subclass for training.")
    def predict(self, inputdata):
        raise NotImplementedError("Check subclass for predicting.")

In [2]:
#this class is responsible for kmeans clustering unsupervised
#klusters = the amount of clusters for the kmeans 
#maxiterations = is the amount of times the computer can reiterate through
#the code until it automatically stops 
#conver = when finidng centroids, and the difference between the last
#centroid is lower than this threshold, finding centroids stop. 
#cent = the centroids, rn we have none
class kmclus(MachineLearningALgos):
    def __init__(self, klusters, maxiterations=100, conver=1e-4):
        self.klusters = klusters
        self.maxiterations = maxiterations
        self.conver = conver
        self.cent = None
        
        
        
        
        
    #this is a definition for finding the centroids in the data
    #isitbool sees if the data is priperly labeled 
    #goes through each of the clusters and find centroids 
    #makes sure there are outputs and data lables 
    #cluspoint is the data point of cluster, and only current cluster 
    #info storeed when at it. then centroid is stored
    def findthecent(self, inputdata, isitbool=None, output=None):
        self.cent = np.zeros((self.klusters, inputdata.shape[1]))
        for klus in range(self.klusters):
            if isitbool is not None and output is not None:
                cluspoint = inputdata[isitbool & (output == klus)]
                if len(cluspoint) > 0:
                    self.cent[klus] = np.mean(cluspoint, axis=0)
                    print(f"randomly assigning centroid for cluster {klus}")
                    continue
            #use np random in case there is no data labels and it will be assigned randomly
            self.cent[klus] = inputdata[np.random.choice(inputdata.shape[0])]
        print("found the centroids")
        
    
    #make clusters based on the centroids we found in findthecent
    #dist = the euc distance between centroids and datapoints on normalizaed data
    #calculatiosn done with help of numpy 
    def theclusters(self, inputdata):
        #make a place for the cluster assignments to go
        assklus= []
        for pointofdata in inputdata:
            mindist = float('inf')
            nearbycent = None
            for i, centroid in enumerate(self.cent):
                dist=np.linalg.norm(pointofdata - centroid)
                if dist < mindist:
                    mindist = dist
                    nearbycent = i 
            assklus.append(nearbycent)
        return np.array(assklus)
    
    
    
    #lbls = stores where all the data points were assigned to includes the index of the cluster 
    #finds newer centroids of each cluster by finding the mean of all of its data points
    #stores them in newcent, and if there is no info then it is done randomly 
    def calcthecurrcent(self, inputdata, lbls):
        newcent= np.zeros((self.klusters, inputdata.shape[1]))
        for klus in range(self.klusters):
            cluspoint = inputdata[lbls == klus]
            if len(cluspoint) > 0:
                centsum = np.zeros(inputdata.shape[1])
                for dpoint in cluspoint:
                    centsum += dpoint
                newcent[klus] = centsum / len(cluspoint)
            else:
                print("error, no info must be done randomly.")
                newcent[klus] = inputdata[np.random.choice(inputdata.shape[0])]
        print(f"the new centroids are {newcent}")
        return newcent
    
    
    #checks if reacked the minimum amount on convergence to stop initializing the centroids. 
    #if less than the defined then it stops
    #it also stops it once the max iterations occurs. this is applied to bot the kmeans algos
    #it starts off as true because it is implies that it is 
    def isitconverged(self, oldcent, newcent, iteri):
        for i in range(len(oldcent)):
            #find euc dist 
            itdiffers= np.sqrt(np.sum((newcent[i]-oldcent[i])**2))
            if itdiffers >= self.conver:
                return False #false means it is NOT converged 
        if iteri >= self.maxiterations -1:
            print("max iterations completed stopping iterations now.")
            return True #max iterations ends before convergence, convergence can keep going 
        return True
        
        

    #now we train the mdoel definition , make sure its numpy array and make sure hparamaters make sense
    def train(self, inputdata, output=None):
        inputdata = np.array(inputdata)
        if not (self.klusters > 0 and isinstance(self.klusters, int)):
            raise ValueError("has to be a postive number")
        if not (self.conver >= 0):
            raise ValueError("cant be a negative number")
        if not (self.maxiterations > 0 and isinstance(self.maxiterations, int)):
            raise ValueError("has to be a postive number")
        
        #now we randomize the starting clusters 
        print("random clusters are being assigned right now")
        iniklus = np.random.choice(range(self.klusters), size=inputdata.shape[0])
        inputdata = np.hstack([inputdata, iniklus[:, None]])
        feats = inputdata.shape[1]-1
        
        #find centroids until converegence or max iteraitons 
        #keps going until it finds the best centroid
        self.cent = np.zeros((self.klusters, feats))
        lbls = iniklus
        for iteri in range(self.maxiterations):
            print(f"\n on iteration number {iteri + 1}")
            #assigning clusters
            lbls = self.theclusters(inputdata[:,:-1])
            print(f"on iteration {iteri + 1} and the assignments are {lbls}")
            #calculating the centroids 
            newcent = self.calcthecurrcent(inputdata[:,:-1], lbls)
            print(f"on iteration {iteri + 1}and the new centroids are {newcent}")

            #checks convergence ansd max iterations and if its done it stops. 
            if self.isitconverged(self.cent, newcent, iteri):
                print("converged or max iterations, stopping now.")
                break
            self.cent = newcent
        return self.cent, lbls
#now prediction happens and the clusters datapoints fall into are predicted. 
#predicted with the clusters def and input data is given. 
#semi sup usees this predict too 

    def predict(self, inputdata):
        if self.cent is None:
            raise ValueError("train model before prediction")
        return self.theclusters(np.array(inputdata))

In [3]:
#this is semisupervised version of kmeans and this can handle null data.
class semikmclus(kmclus):
    def train(self, inputdata, output=None):
        if output is None:
            raise ValueError("data must be labeled for semi supervised kmeans. can be null.")
        
        #input and output data are arrays
        inputdata = np.array(inputdata)
        output = np.array(output)
        
        #do this again : make sure hparamaters make sense
        if not (self.klusters > 0 and isinstance(self.klusters, int)):
            raise ValueError("has to be a postive number")
        if not (self.conver >= 0):
            raise ValueError("cant be a negative number")
        if not (self.maxiterations > 0 and isinstance(self.maxiterations, int)):
            raise ValueError("has to be a postive number")
        
        #make sure data is labeled with boolean true or false 
        isitbool = ~np.isnan(output)
        itsnotbool = np.isnan(output)
        
        #find centroids
        self.findthecent(inputdata[:,:-1], isitbool=isitbool, output=output)
        
        #assign clusters 
        clusassign = np.zeros(inputdata.shape[0], dtype=int)
        clusassign[isitbool] = output[isitbool].astype(int)
        
        #do this again : find centroifds 
        for iteri in range(self.maxiterations):
            print(f"\n on iteration number {iteri + 1}")
            
            #need to remove the last column 
            updateddata=inputdata[:,:-1] 
            
            #unlabeled data are assigned to clusters 
            clusassign[itsnotbool] = self.theclusters(updateddata[itsnotbool])
            print(f"cluster assignments are {clusassign}")
            
            #new centroids found 
            newcent = self.calcthecurrcent(updateddata, clusassign)
            print(f"newest centroids are \n{newcent}")
            
            #see if data is convereged or hit max iteratiosn 
            if self.isitconverged(self.cent, newcent, iteri):
                print("converged or max iterations, stopping now.")
                break
            self.cent = newcent
        return self.cent, clusassign

In [4]:
#question 3 

#do PCA 
#numcom = number of components 
#pcom is the principle components 
class princompanaly(MachineLearningALgos):
    def __init__(self, numcom):
        self.numcom = numcom
        self.meanz = None
        self.standev = None
        self.pcom = None
#edit the data normalizing it and standard dev and mean (standardizing) 
    def standdata(self, inputdata):
        print("standardize the data before doing pca")
        self.meanz = np.mean(inputdata, axis=0)
        self.standev = np.std(inputdata, axis=0)
        return (inputdata - self.meanz) / self.standev
#this definition is responsible for undoing the standarization of the dataset after PCA
#standdataset is the dataset that is standardized 
    def notstanddata(self, standdataset):
        print("making dataset back to original (removing standardizatiosn)")
        return (standdataset * self.standev) + self.meanz
#now we train the PCA model 
#input data and labels are defined but labels are not used in PCA 
#make sure the number of components and size is valid 
    def train(self, inputdata, output=None):
        if not (self.numcom > 0 and self.numcom <= min(inputdata.shape)):
            raise ValueError("number of componenets is either neg or not right shape. recheck.")
#use the standdata def to standardize and define the dataset for pca
        standdataset = self.standdata(inputdata)
#now do svd 
        print("starting single value decomp")
#now we have the vecorts but they arent used in pca 
#we have the single values which is sqrt of eignvalues 
#we also have the transposed vecors 
#full matricies is in numpy to say that matriz is going to be reduced 
        svdvec, singval, transpvec = np.linalg.svd(standdataset, full_matrices=False)
#now we do matrix verification 
        covmatx = np.cov(standdataset, rowvar=False)
        eignvec = transpvec.T
        appcov = eignvec @ np.diag(singval**2 / (standdataset.shape[0] - 1)) @ eignvec.T
#check if tansposed vec have the eighn
        if np.allclose(covmatx, appcov, atol=1e-6):
            print("eighnvecors are in the matrix")
        else:
            raise ValueError("eighnvectors aare not in the maxrix check whats going on")
#numcom defined says how many pc's there are going to be and picks the top ones for the amount deifned 
        print(f"based on numcom there are {self.numcom} pc(s) being picked right now")
        self.pcom = transpvec[:self.numcom]
#now data is transformed with pca on input data 
    def predict(self, inputdata):
        if self.pcom is None:
            raise ValueError("model has to be trained before predicting")
#use the standdata def to standardize and define the dataset for pca
        standdataset = self.standdata(inputdata)
#compress for pca transformation 
        print("data is compressing")
        compresseddata = self.pcom @ standdataset.T
#standardize and decompress the data 
        print("data being rebuilt")
        decompstandata = self.pcom.T @ compresseddata
#unstandardize the data back to original value scale 
        datadecompfin = self.notstanddata(decompstandata.T)
        return datadecompfin

In [5]:
#we use main to run the code and define the input data 
def main():
#now doing unsup kmean clustering
    print("\n now doing unsupervised k means clustering")
    inputdata = np.array([
        [1.0, 2.0],
        [1.5, 1.8],
        [5.0, 8.0],
        [8.0, 8.0],
        [1.0, 0.6],
        [9.0, 11.0]
    ])
    kmeanz = kmclus(klusters=2, maxiterations=100, conver=1e-4)
    cent, labelz = kmeanz.train(inputdata)
    print("the centroids in the data are \n", cent)
    print("the datapoints are assigned to these clusters \n", labelz)
#use predict to find what clusters belong to which new data  
    newdataaaa = np.array([[0.5, 2.0], [6.0, 8.0]])
    predis = kmeanz.predict(newdataaaa)
    print("the new clusters are predicted to be assign to \n", predis)
#doing semisup kmean clustering
    print("\n now doing semi supervised k means clustering")
#data with some that arre not labeled 
    inputdata = np.array([
        [1.0, 2.0],
        [1.5, 1.8],
        [5.0, 8.0],
        [8.0, 8.0],
        [1.0, 0.6],
        [9.0, 11.0]
    ])
    labelz = np.array([0, 0, np.nan, np.nan, 0, np.nan])
    semikmeanz = semikmclus(klusters=2, maxiterations=100, conver=1e-4)
    cent, labelz = semikmeanz.train(inputdata, output=labelz)
    print("the centroids in the data are \n", cent)
    print("the datapoints are assigned to these clusters \n", labelz)
#doing pca
    print("\n now doing pca")
    inputdata = np.array([
        [1.0, 2.0],
        [1.5, 1.8],
        [5.0, 8.0],
        [8.0, 8.0],
        [1.0, 0.6],
        [9.0, 11.0]
    ])
#define num of compotnents and train the data 
    numcom = 2 
    doingpca = princompanaly(numcom=numcom)
    doingpca.train(inputdata)
#do rpedictions 
    print("the original data \n", inputdata)
    tansfdata = doingpca.predict(inputdata)
    print("data after the pca \n", tansfdata)
if __name__ == "__main__":
    main()


 now doing unsupervised k means clustering
random clusters are being assigned right now

 on iteration number 1
on iteration 1 and the assignments are [0 0 0 0 0 0]
error, no info must be done randomly.
the new centroids are [[ 4.25        5.23333333]
 [ 9.         11.        ]]
on iteration 1and the new centroids are [[ 4.25        5.23333333]
 [ 9.         11.        ]]

 on iteration number 2
on iteration 2 and the assignments are [0 0 0 1 0 1]
the new centroids are [[2.125 3.1  ]
 [8.5   9.5  ]]
on iteration 2and the new centroids are [[2.125 3.1  ]
 [8.5   9.5  ]]

 on iteration number 3
on iteration 3 and the assignments are [0 0 1 1 0 1]
the new centroids are [[1.16666667 1.46666667]
 [7.33333333 9.        ]]
on iteration 3and the new centroids are [[1.16666667 1.46666667]
 [7.33333333 9.        ]]

 on iteration number 4
on iteration 4 and the assignments are [0 0 1 1 0 1]
the new centroids are [[1.16666667 1.46666667]
 [7.33333333 9.        ]]
on iteration 4and the new centro